In [3]:
import pandas as pd
import numpy as np

df=pd.read_csv("Fraud.csv")
print(df)

         step      type     amount     nameOrig  oldbalanceOrg  \
0           1   PAYMENT    9839.64  C1231006815      170136.00   
1           1   PAYMENT    1864.28  C1666544295       21249.00   
2           1  TRANSFER     181.00  C1305486145         181.00   
3           1  CASH_OUT     181.00   C840083671         181.00   
4           1   PAYMENT   11668.14  C2048537720       41554.00   
...       ...       ...        ...          ...            ...   
1048570    95  CASH_OUT  132557.35  C1179511630      479803.00   
1048571    95   PAYMENT    9917.36  C1956161225       90545.00   
1048572    95   PAYMENT   14140.05  C2037964975       20545.00   
1048573    95   PAYMENT   10020.05  C1633237354       90605.00   
1048574    95   PAYMENT   11450.03  C1264356443       80584.95   

         newbalanceOrig     nameDest  oldbalanceDest  newbalanceDest  isFraud  \
0             160296.36  M1979787155            0.00            0.00        0   
1              19384.72  M2044282225         

In [4]:
# Load only a subset of the data
df_sample = df.sample(frac=0.1, random_state=42)  # use 10% of data

# Drop unnecessary columns
df_sample = df_sample.drop(columns=['newbalanceOrig', 'nameOrig', 'nameDest'])

# Encode 'type' column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_sample['type'] = le.fit_transform(df_sample['type'])

# Split features and target
X = df_sample.drop(columns=['isFraud'])
y = df_sample['isFraud']


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load full dataset
df = pd.read_csv("Fraud.csv")

# Drop columns that are high-cardinality or redundant
df = df.drop(columns=['newbalanceOrig', 'nameOrig', 'nameDest'])

# Encode 'type' column
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])

# Prepare features and target
X = df.drop(columns=['isFraud'])
y = df['isFraud']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# Train model
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    314230
           1       1.00      0.74      0.85       343

    accuracy                           1.00    314573
   macro avg       1.00      0.87      0.93    314573
weighted avg       1.00      1.00      1.00    314573



In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

X = add_constant(X)
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)


          Feature        VIF
0           const   6.714066
1            step   1.008467
2            type   1.177533
3          amount   1.404912
4   oldbalanceOrg   1.143371
5  oldbalanceDest  31.180263
6  newbalanceDest  32.553350
7  isFlaggedFraud        NaN


C:\Users\vikra\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


In [8]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Step 1: Load data
df = pd.read_csv("Fraud.csv")

# Step 2: Drop high-cardinality or identifier columns
df = df.drop(columns=["nameOrig", "nameDest", "newbalanceOrig"])

# Step 3: Handle missing values (check if any)
print("Missing values:\n", df.isnull().sum())

# Step 4: Encode 'type' column (categorical)
le = LabelEncoder()
df['type'] = le.fit_transform(df['type'])

# Step 5: Basic outlier filtering (optional, keep amounts > 0)
df = df[df['amount'] > 0]

# Step 6: Define features and target
X = df.drop(columns=["isFraud"])
y = df["isFraud"]

# Step 7: Check and remove multicollinearity using VIF
X_const = add_constant(X)
vif_df = pd.DataFrame()
vif_df["Feature"] = X_const.columns
vif_df["VIF"] = [variance_inflation_factor(X_const.values, i) for i in range(X_const.shape[1])]
print("\nVariance Inflation Factors:\n", vif_df)

# Remove variables with VIF > 10
high_vif_features = vif_df[vif_df["VIF"] > 10]["Feature"]
high_vif_features = high_vif_features[high_vif_features != "const"]
X = X.drop(columns=high_vif_features)

# Step 8: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

# Step 9: Train Logistic Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Step 10: Evaluate Model
y_pred = model.predict(X_test)

print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Missing values:
 step              0
type              0
amount            0
oldbalanceOrg     0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64


C:\Users\vikra\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss



Variance Inflation Factors:
           Feature        VIF
0           const   6.714066
1            step   1.008467
2            type   1.177533
3          amount   1.404912
4   oldbalanceOrg   1.143371
5  oldbalanceDest  31.180263
6  newbalanceDest  32.553350
7  isFlaggedFraud        NaN

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    314230
           1       0.74      0.08      0.14       343

    accuracy                           1.00    314573
   macro avg       0.87      0.54      0.57    314573
weighted avg       1.00      1.00      1.00    314573


Confusion Matrix:
 [[314221      9]
 [   317     26]]
